In [ ]:
"""
Close MNQ Position and Cancel All Orders
This will:
1. Close any open MNQ position
2. Cancel all pending orders (including TP and SL brackets)
"""
import os
import requests
from dotenv import load_dotenv

load_dotenv()

USERNAME = os.getenv("TOPSTEP_USERNAME")
KEY = os.getenv("TOPSTEP_KEY")
API_BASE = "https://api.topstepx.com/api"

print("=" * 60)
print("CLOSE POSITION & CANCEL ORDERS - MNQ")
print("=" * 60)

# Authenticate
print("\n[1/4] Authenticating...")
auth_response = requests.post(
    f"{API_BASE}/Auth/loginKey",
    json={"userName": USERNAME, "apiKey": KEY},
    headers={"accept": "text/plain", "Content-Type": "application/json"},
    timeout=15
)

if auth_response.status_code != 200:
    print(f"❌ Auth failed")
    exit(1)

token = auth_response.json()["token"]
print(f"✅ Authenticated")

headers = {
    "Authorization": f"Bearer {token}",
    "accept": "text/plain",
    "Content-Type": "application/json"
}

# Get Account
print("\n[2/4] Getting account...")
search_response = requests.post(
    f"{API_BASE}/Account/search",
    json={"onlyActiveAccounts": True},
    headers=headers,
    timeout=15
)

account = search_response.json()["accounts"][0]
account_id = account["id"]
print(f"✅ Account: {account['name']}")
print(f"   Balance: ${account.get('balance', 0):,.2f}")

# Close Position
print("\n[3/4] Closing MNQ position...")
close_payload = {
    "accountId": account_id,
    "contractId": "CON.F.US.MNQ.H26"
}

close_response = requests.post(
    f"{API_BASE}/Position/closeContract",
    json=close_payload,
    headers=headers,
    timeout=15
)

print(f"Status Code: {close_response.status_code}")
close_result = close_response.json()
print(f"Response: {close_result}")

if close_result.get('success'):
    print(f"✅ Position closed successfully!")
else:
    print(f"⚠️ Position close response: {close_result.get('errorMessage')}")

# Get and Cancel All Open Orders
print("\n[4/4] Cancelling all open orders...")

search_params = {
    "accountId": account_id
}

orders_response = requests.post(
    f"{API_BASE}/Order/searchOpen",
    json=search_params,
    headers=headers,
    timeout=15
)

print(f"Order search status: {orders_response.status_code}")

if orders_response.status_code == 200:
    try:
        orders_data = orders_response.json()
        
        if not orders_data.get('success'):
            print(f"⚠️ Order search failed: {orders_data.get('errorMessage')}")
        else:
            open_orders = orders_data.get("orders", [])
            
            if not open_orders:
                print("ℹ️ No open orders found")
            else:
                print(f"Found {len(open_orders)} open order(s)")
                
                cancelled_count = 0
                for order in open_orders:
                    order_id = order.get("id")
                    order_side = "BUY" if order.get("side") == 0 else "SELL"
                    order_size = order.get("size", "?")
                    order_type_map = {1: "Limit", 2: "Market", 4: "Stop", 5: "TrailingStop"}
                    order_type = order_type_map.get(order.get("type"), "Unknown")
                    contract = order.get("contractId", "?")
                    stop_price = order.get("stopPrice")
                    limit_price = order.get("limitPrice")
                    
                    print(f"\n  Cancelling Order ID {order_id}")
                    print(f"    Contract: {contract}")
                    print(f"    Type: {order_type}, Side: {order_side}, Size: {order_size}")
                    if stop_price:
                        print(f"    Stop Price: {stop_price}")
                    if limit_price:
                        print(f"    Limit Price: {limit_price}")
                    
                    cancel_payload = {
                        "accountId": account_id,
                        "orderId": order_id
                    }
                    
                    cancel_response = requests.post(
                        f"{API_BASE}/Order/cancel",
                        json=cancel_payload,
                        headers=headers,
                        timeout=15
                    )
                    
                    cancel_result = cancel_response.json()
                    if cancel_result.get('success'):
                        print(f"  ✅ Order {order_id} cancelled")
                        cancelled_count += 1
                    else:
                        print(f"  ❌ Failed: {cancel_result.get('errorMessage')}")
                
                print(f"\n✅ Successfully cancelled {cancelled_count}/{len(open_orders)} orders")
    except Exception as e:
        print(f"❌ Error processing orders: {e}")
        print(f"Response text: {orders_response.text}")
else:
    print(f"❌ Order search failed")
    print(f"Response: {orders_response.text}")

print("\n" + "=" * 60)
print("Summary:")
print("- Position: CLOSED")
print("- All open orders: CANCELLED")
print("=" * 60)